# Group Project Proposal: Home Price Regression

## 1. Introduction

#### 1.1 Background Information

Housing prices vary according to location, size of the house, size of the lot, and many other factors. As such, it can be difficult to predict what price one should sell their house for or what price one should offer for a house. This project is built to help figure out these values and the variables that affect housing prices.

#### 1.2 Objective Question

Which real estate factors contribute to most accurately predicting the selling price of a home? Can we build a regression model to accurately predict home prices based on given input real estate factors?

#### 1.3 Dataset Description

The dataset we will be using has data for ~1500 house sales in the city of Ames, Iowa from 2006-2010. The response variable in this dataset is the sale price of each home. There are approximately 80 descriptive real estate factors (including Living Area, number of bedrooms and bathrooms, and lot size) for each observation.

In [1]:
#Place fun house picture here if possible!

## 2. Preliminary exploratory data analysis

#### 2.1 Reading in data

In [1]:
#TODO Check which packages are included in base R
install.packages('gsheet')
library(tidyverse)
#library(cowplot)
#library(scales)
library(gsheet)
library(repr)
data <- gsheet2tbl('docs.google.com/spreadsheets/d/1nNlzfwXkHVk2i946pgf3247KT2vqCDfHZjrBjT-losg/edit?usp=sharing')

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



#### 2.2 Clean and wrangle data

In [4]:
#Raw data is already in tidy data format

#### 2.3 Tabular Summary of Data

In [7]:
#Using only training data, summarize the data in at least one table (this is exploratory data analysis). 
#An example of a useful table could be one that reports the number of observations in each class, 
#the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 

#### 2.4 Visual Summary of Data

In [8]:
#Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis).
#An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your 
#analysis.

## 3. Methods

In [11]:
#Explain how you will conduct either your data analysis and which variables/columns you will use. 

#Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. 
#For each variable think: is this a useful variable for prediction?

## 4. Expected outcomes and significance

#### 4.1 Expected findings

In [12]:
# What do you expected to find?

#### 4.2 Potential impact

In [13]:
#What impact could such findings have?

#### 4.3 Potential future questions

In [15]:
#What future questions could this lead to?